In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-09-10 12:50:48 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-09-10 12:50:49 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-09-10 12:50:49 __main__ INFO     transformers.__version__='4.55.3'


In [3]:
from src.utils.training_utils import get_device_map

model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
# model_key = "meta-llama/Llama-3.1-70B-Instruct"
# model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

2025-09-10 12:50:52 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-09-10 12:50:52 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-09-10 12:50:52 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-09-10 12:50:52 wandb.docker.auth DEBUG    No config file found


In [4]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
    attn_implementation="eager",
)

2025-09-10 12:50:53 src.models WARNING  meta-llama/Llama-3.2-3B not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-09-10 12:50:53 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2025-09-10 12:50:53 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/config.json HTTP/1.1" 200 0
2025-09-10 12:50:53 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-09-10 12:50:53 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/models/meta-llama/Llama-3.2-3B/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2025-09-10 12:50:58 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-09-10 12:50:58 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0
2025-09-10 12:50:58 src.models INFO     loaded model <meta-llama/Llama-3.2-3B> | size: 6127.834 MB | dtype: torch.bfloat16 | device: cuda:0


In [5]:
from src.selection.data import SelectOneTask, CountingTask, YesNoTask, SelectFirstTask
from src.selection.data import SelectionSample, CountingSample, YesNoSample

#################################################################################
# TASK_CLS = SelectOneTask
# TASK_CLS = CountingTask
TASK_CLS = YesNoTask
prompt_template_idx = 0
N_DISTRACTORS = 5
OPTION_STYLE = "single_line"
#################################################################################

select_task = TASK_CLS.load(
    path=os.path.join(
        env_utils.DEFAULT_DATA_DIR, 
        "selection", 
        # "profession.json"
        # "nationality.json"
        "objects.json"
    )
)

print(select_task)

YesNoTask: (different objects)
Categories: fruit(15), vehicle(15), furniture(15), animal(15), music instrument(15), clothing(15), electronics(15), sport equipment(15), kitchen appliance(15), vegetable(14), building(15), office supply(15), bathroom item(15), flower(15), tree(15), jewelry(15)


In [6]:
from src.functional import predict_next_token, generate_with_patch

sample = select_task.get_random_sample(
    mt=mt, 
    category="fruit",
    filter_by_lm_prediction=False,
    prompt_template_idx=1,
    n_options=5,
)
print(f'"{sample.prompt()}" >> {mt.tokenizer.decode(sample.ans_token_id)}')

gen = generate_with_patch(
    mt=mt, 
    inputs=sample.prompt(),
    n_gen_per_prompt=1,
    remove_prefix=True,
    max_new_tokens=20,
    do_sample=False,
)
print(f"Generation:\"{gen[0]}\"") # expect sample.count

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
You have set `compile_config`, but we are unable to meet the criteria for compilation. Compilation will be skipped.


"Dress, Cauliflower, Charm, Daffodil, Ottoman
Answer 'Yes' or 'No': Is there a fruit in the list above?
Answer:" >>  No
Generation:" Yes, there is a fruit in the list above.
The fruit is the cauliflower.
The cauliflower is"


In [7]:
sample.prediction

In [8]:
from src.selection.data import get_counterfactual_samples_interface

counterfactual_sampler = get_counterfactual_samples_interface[select_task.task_name]

print(counterfactual_sampler)

patch_sample, clean_sample = counterfactual_sampler(
    mt=mt,
    task=select_task,
    patch_category="fruit",
    clean_category="vehicle",
    filter_by_lm_prediction=False,
    prompt_template_idx=1,
    option_style=OPTION_STYLE,
    # distinct_options=True,
    n_options=6
)

# patch_sample.default_option_style = "single_line"
# clean_sample.default_option_style = "numbered"

print("-" * 100)
print(patch_sample.prompt(), ">>", mt.tokenizer.decode(patch_sample.ans_token_id))
print(clean_sample.prompt(), ">>", mt.tokenizer.decode(clean_sample.ans_token_id))

<function get_counterfactual_samples_within_yes_no_task at 0x7eff6412d800>
----------------------------------------------------------------------------------------------------
Skirt, Keyboard, Lettuce, Dumbbell, Cherry, Scooter
Answer 'Yes' or 'No': Is there a fruit in the list above?
Answer: >>  Yes
Museum, Hockey stick, Bus, Chair, Tractor, Food processor
Answer 'Yes' or 'No': Is there a vehicle in the list above?
Answer: >>  Yes


In [39]:
patch_sample.prediction, clean_sample.prediction

(OrderedDict([(9220,
               (3,
                PredictedToken(token=' Two', prob=0.2333984375, logit=21.5, token_id=9220, metadata=None))),
              (14853,
               (8,
                PredictedToken(token=' Three', prob=0.003326416015625, logit=17.25, token_id=14853, metadata=None))),
              (13625,
               (17,
                PredictedToken(token=' Four', prob=0.00045013427734375, logit=15.25, token_id=13625, metadata=None))),
              (3861,
               (19,
                PredictedToken(token=' One', prob=0.000423431396484375, logit=15.1875, token_id=3861, metadata=None))),
              (21594,
               (42,
                PredictedToken(token=' Five', prob=7.82012939453125e-05, logit=13.5, token_id=21594, metadata=None))),
              (19198,
               (75,
                PredictedToken(token=' Six', prob=2.8848648071289062e-05, logit=12.5, token_id=19198, metadata=None))),
              (18811,
               (117,
    